## Login to Tanzu Mission Control

In [ ]:
source ../config.sh

tmc login --no-configure --name tanzu-demo

## Create clusters

In [ ]:
source ../config.sh

tmc cluster create --account-name ${ACCOUNT_NAME} --ssh-key-name ${SSH_KEY_NAME} --region ${REGION} --worker-node-count 3 --instance-type m5.xlarge --group ${GROUP} --name ${CICD_CLUSTER}
tmc cluster create --account-name ${ACCOUNT_NAME} --ssh-key-name ${SSH_KEY_NAME} --region ${REGION} --worker-node-count 3 --instance-type m5.xlarge --group ${GROUP} --name ${ALPHA_CLUSTER}

## Monitor creation progress

In [ ]:
tmc cluster list --group bush

## Add clusters to kubeconfig file

In [ ]:
source ../config.sh

KUBECONFIG_CICD=`mktemp` && \
tmc cluster provisionedcluster kubeconfig get ${CICD_CLUSTER} >> ${KUBECONFIG_CICD} && \
KUBECONFIG_ALPHA=`mktemp` && \
tmc cluster provisionedcluster kubeconfig get ${ALPHA_CLUSTER} >> ${KUBECONFIG_ALPHA}

if [ $? -eq 0 ]; then
    KUBECONFIG=${KUBECONFIG_CICD}:${KUBECONFIG_ALPHA}:${KUBECONFIG_BRAVO} kubectl config view --merge --flatten > ~/.kube/config

    rm ${KUBECONFIG_CICD}
    rm ${KUBECONFIG_ALPHA}
fi

## Update cluster permissions

In [ ]:
source ../config.sh

kubectl config use-context ${CICD_CLUSTER}
kubectl create clusterrolebinding privileged-role-binding --clusterrole=vmware-system-tmc-psp-privileged --group=system:authenticated

echo
kubectl config use-context ${ALPHA_CLUSTER}
kubectl create clusterrolebinding privileged-role-binding --clusterrole=vmware-system-tmc-psp-privileged --group=system:authenticated

## Delete clusters

In [ ]:
source ../config.sh

tmc cluster delete ${CICD_CLUSTER}
tmc cluster delete ${ALPHA_CLUSTER}

# Install [Argo CD](https://argoproj.github.io/argo-cd/)

In [ ]:
source ../config.sh

kubectl config use-context ${CICD_CLUSTER}

echo
echo 'Create Argo CD namespace...'
kubectl create namespace argocd

echo
echo 'Install Argo CD on the CI/CD cluster...'
kubectl -n argocd apply -f https://raw.githubusercontent.com/argoproj/argo-cd/stable/manifests/install.yaml

echo
echo 'Fetch a Load Balanced IP for the Argo CD server...'
kubectl -n argocd patch svc argocd-server -p '{"spec": {"type": "LoadBalancer"}}'

### Wait for load balancer to be provisioned...

In [ ]:
source ../config.sh

kubectl config use-context ${CICD_CLUSTER}

kubectl -n argocd get svc argocd-server
nslookup `kubectl -n argocd get svc/argocd-server -o=jsonpath='{.status.loadBalancer.ingress[0].hostname}'`

### Finish configuring Argo CD

In [ ]:
source ../config.sh

kubectl config use-context ${CICD_CLUSTER}

ARGOCD_USERNAME=admin
ARGOCD_PASSWORD=`kubectl -n argocd get pods -l app.kubernetes.io/name=argocd-server -o name | cut -d'/' -f 2`

echo
echo 'Store the Argo CD host name...'
ARGOCD_HOST=`kubectl -n argocd get svc/argocd-server -o json | jq '.status.loadBalancer.ingress[0].hostname' -j`

argocd login ${ARGOCD_HOST} --name argocd --username ${ARGOCD_USERNAME} --password ${ARGOCD_PASSWORD} --insecure

argocd cluster add ${ALPHA_CLUSTER}

echo
echo "Argo CD Host: $ARGOCD_HOST"
echo "Username: $ARGOCD_USERNAME"
echo "Password: $ARGOCD_PASSWORD"

# Install [Kubeapps](https://kubeapps.com/)

### Create service accounts

In [ ]:
source ../config.sh

kubectl config use-context ${ALPHA_CLUSTER}

echo
kubectl create namespace kubeapps
kubectl create -n kubeapps serviceaccount kubeapps-operator
kubectl create clusterrolebinding kubeapps-operator --clusterrole=cluster-admin --serviceaccount=kubeapps:kubeapps-operator

### Configure Argo CD to deploy Kubeapps

In [ ]:
source ../config.sh

ALPHA_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${ALPHA_CLUSTER} | yq r - 'clusters[0].cluster.server'`

kubectl config use-context ${CICD_CLUSTER}

echo
echo 'Configure the Kubeapps Helm chart in Argo CD...'
cat ../kubernetes/cicd/argocd/kubeapps.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/alpha/" | kubectl apply -n argocd -f -

### Get Kubeapps endpoints & login tokens

In [ ]:
source ../config.sh

kubectl config use-context ${ALPHA_CLUSTER}

echo
echo 'Kubeapps on Alpha cluster:'
kubectl get -n kubeapps -o json service kubeapps-alpha | jq '.status.loadBalancer.ingress[0].hostname' -j

echo
echo
echo 'Alpha cluster login token:'
kubectl get -n kubeapps secret $(kubectl get -n kubeapps serviceaccount kubeapps-operator -o jsonpath='{.secrets[].name}') -o go-template='{{.data.token | base64decode}}' && echo

## Configure Argo CD to deploy Acme Fitness

In [ ]:
source ../config.sh

kubectl config use-context ${ALPHA_CLUSTER}

kubectl create secret generic cart-redis-pass --from-literal=password=password123
kubectl create secret generic catalog-mongo-pass --from-literal=password=password123
kubectl create secret generic order-postgres-pass --from-literal=password=password123
kubectl create secret generic users-mongo-pass --from-literal=password=password123
kubectl create secret generic users-redis-pass --from-literal=password=password123

ALPHA_CLUSTER_URL=`tmc cluster provisionedcluster kubeconfig get ${ALPHA_CLUSTER} | yq r - 'clusters[0].cluster.server'`

kubectl config use-context ${CICD_CLUSTER}

echo
echo 'Configure the applications in Argo CD...'
cat ../kubernetes/cicd/argocd/acme-fitness.yml | sed "s/CLUSTER_URL/${ALPHA_CLUSTER_URL//\//\\/}/" | sed "s/ENVIRONMENT/alpha/" | kubectl apply -n argocd -f -